In [1]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 5

dev = qml.device('default.mixed', wires=n_qubits)

inputs = np.ones(5, requires_grad=False)
weights = np.ones((3,3,5,3), requires_grad=True)
blocks = 3
p = 0.01

@qml.qnode(dev, interface="tf", diff_method="best")
def qnode(inputs, weights):
    for i in range(blocks):
        qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
        qml.templates.StronglyEntanglingLayers(weights[i], wires=range(n_qubits))
        qml.DepolarizingChannel(p, wires=0)
        qml.DepolarizingChannel(p, wires=1)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (3,3,5,3)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2)
qlayer(inputs)

2023-05-26 19:44:56.433992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 19:45:04.665854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.0789994 ,  0.14434064, -0.0303331 , -0.03299901, -0.02312417])>

In [4]:
print(weights[0][0])

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [5]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def circuit(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(4))
    return qml.expval(qml.PauliZ(0))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=4)
weights = np.random.random(size=shape)

In [6]:
print(qml.draw(circuit, expansion_strategy="device")(weights))

0: ──Rot(0.83,0.27,0.88)─╭●───────╭X──Rot(0.13,0.13,0.97)─╭●────╭X────┤  <Z>
1: ──Rot(0.15,0.76,0.75)─╰X─╭●────│───Rot(0.81,0.07,0.65)─│──╭●─│──╭X─┤     
2: ──Rot(0.03,0.13,0.63)────╰X─╭●─│───Rot(0.43,0.40,0.95)─╰X─│──╰●─│──┤     
3: ──Rot(0.26,0.38,0.83)───────╰X─╰●──Rot(0.42,0.44,0.51)────╰X────╰●─┤     


In [7]:
print(shape)

(2, 4, 3)


In [10]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 5

dev = qml.device('default.mixed', wires=n_qubits)

inputs = np.ones(5, requires_grad=False)
weights = np.ones((3,5,3), requires_grad=True)
#blocks = 3
p = 0.01

@qml.qnode(dev, interface="tf", diff_method="best")
def qnode(inputs, weights):
    #for i in range(blocks):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    qml.DepolarizingChannel(p, wires=0)
    qml.DepolarizingChannel(p, wires=1)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (3,5,3)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2)
qlayer(inputs)

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.05408433, 0.05035527, 0.12459255, 0.03712322, 0.27445235])>

In [19]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit(feature_vector):
    qml.AngleEmbedding(features=feature_vector, wires=range(3), rotation='X')
    qml.Hadamard(0)
    return qml.probs(wires=range(3))

X = [1,2,3]

In [20]:
print(qml.draw(circuit, expansion_strategy="device")(X))

0: ──RX(1.00)──H─┤ ╭Probs
1: ──RX(2.00)────┤ ├Probs
2: ──RX(3.00)────┤ ╰Probs
